In [1]:
#%pip install openai -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast

from utils.gpt_label_utils import predict

from utils.instructions import label_instructions_v515
MODEL_ID = "gpt-4.1-2025-04-14"
SYSTEM_PROMPT = label_instructions_v515

In [2]:
df = pd.read_csv("../experiments_501/data/output_dataset.csv")
df.head()

,citing_cluster_id,citing_decision_name,citing_url,citing_opinions,citing_filenames,cited_cluster_id,cited_decision_name,cited_url,cited_name_short,cited_name,cited_name_full,cited_citations,overruled,note,use_full_opinion,filename
0,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88061,Texas v. White (1869),https://www.courtlistener.com/opinion/88061/te...,White,Texas v. White,Texas v. White Et Al.,"['74 U.S. 700', '19 L. Ed. 227', '7 Wall. 700'...",yes,NaN,0,0001.91306_cites_88061.txt
1,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88994,Vermilye & Co. v. Adams Express Co. (1875),https://www.courtlistener.com/opinion/88994/ve...,NaN,Vermilye & Co. v. Adams Express Co.,Vermilye & Co. v. Adams Express Company,"['88 U.S. 138', '22 L. Ed. 609', '21 Wall. 138...",no,NaN,0,0002.91306_cites_88994.txt
2,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],87633,Murray v. Lardner (1865),https://www.courtlistener.com/opinion/87633/mu...,Murray,Murray v. Lardner,Murray v. Lardner,"['69 U.S. 110', '17 L. Ed. 857', '2 Wall. 110'...",no,NaN,0,0003.91306_cites_87633.txt
3,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88240,Texas v. Hardenberg (1869),https://www.courtlistener.com/opinion/88240/te...,Hardenberg,Texas v. Hardenberg,Texas v. Hardenberg,"['77 U.S. 68', '19 L. Ed. 839', '10 Wall. 68',...",no,NaN,0,0004.91306_cites_88240.txt
4,91306,"Morgan v. United States,113 U.S. 476 (1885)",https://www.courtlistener.com/opinion/91306/mo...,"[{'opinion_id': 91306, 'opinion_api': 'https:/...",['91306_010combined.txt'],88693,Huntington v. Texas (1873),https://www.courtlistener.com/opinion/88693/hu...,Huntington,Huntington v. Texas,Huntington v. Texas; Texas v. Huntington,"['83 U.S. 402', '21 L. Ed. 316', '16 Wall. 402...",no,NaN,0,0005.91306_cites_88693.txt


In [3]:
citing_dict = df[['citing_cluster_id', 'citing_decision_name', 'citing_opinions', 'citing_filenames']].drop_duplicates().set_index('citing_cluster_id')[['citing_decision_name', 'citing_filenames']].to_dict(orient='index')
len(citing_dict.keys())

133

In [4]:
cited_dict = {}

citations_df = df[['citing_cluster_id', 'cited_cluster_id',
       'cited_decision_name', 'cited_name_short', 'cited_name',
       'cited_name_full', 'cited_citations']]

for _, row in citations_df.iterrows():
    citing = row['citing_cluster_id']
    cited = row['cited_cluster_id']
    
    # Extract metadata (everything except citing and cited IDs)
    metadata = row.drop(labels=['citing_cluster_id', 'cited_cluster_id']).to_dict()
    
    if citing not in cited_dict:
        cited_dict[citing] = {}
    
    cited_dict[citing][cited] = metadata
    
len(cited_dict.keys())

133

In [5]:
%%time

file_path = "../experiments_501/data/raw_citing_opinions"
raw_results_df, parsed_results_df = predict(citing_dict, cited_dict, model_id=MODEL_ID, system_prompt=SYSTEM_PROMPT, data_folder=file_path)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 1: citing_id 92291
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 2: citing_id 93904
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 3: citing_id 98124
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 4: citing_id 91306
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 5: citing_id 99004
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 6: citing_id 93311
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Processed 7: citing_id 99901
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

CPU times: user 8.66 s, sys: 899 ms, total: 9.56 s
Wall time: 9min 12s


In [6]:
raw_results_df

,citing_cluster_id,model,input_tokens,output_tokens,raw_results
0,92291,gpt-4.1-2025-04-14,6712,690,"{\n ""88699"": {\n ""cited_decision"": ""Osborn..."
1,93904,gpt-4.1-2025-04-14,5241,1003,"{\n ""93354"": {\n ""cited_decision"": ""Robert..."
2,98124,gpt-4.1-2025-04-14,4150,335,"{\n ""96356"": {\n ""cited_decision"": ""Rogers..."
3,91306,gpt-4.1-2025-04-14,11051,1205,"{\n ""88061"": {\n ""cited_decision"": ""Texas ..."
4,99004,gpt-4.1-2025-04-14,7798,1235,"{\n ""95042"": {\n ""cited_decision"": ""Lake S..."
...,...,...,...,...,...
128,118133,gpt-4.1-2025-04-14,23966,2453,"{\n ""112887"": {\n ""cited_decision"": ""Zobre..."
129,118428,gpt-4.1-2025-04-14,12255,825,"{\n ""99610"": {\n ""cited_decision"": ""Evans ..."
130,118317,gpt-4.1-2025-04-14,24923,1934,"{\n ""118011"": {\n ""cited_decision"": ""Semin..."
131,118386,gpt-4.1-2025-04-14,56525,1954,"{\n ""118133"": {\n ""cited_decision"": ""Agost..."


In [7]:
parsed_results_df = parsed_results_df.rename(columns={'treatment': 'prediction'})
parsed_results_df

,citing_cluster_id,cited_cluster_id,cited_decision,prediction,quote,rationale
0,92291,88699,Osborne v. Mobile (1873),Overruled,The state court relies upon the case of Osborn...,The Acting Decision expressly states that the ...
1,92291,89664,Pensacola Telegraph Co. v. Western Union Teleg...,Mentioned,In the case of The Pensacola Telegraph Company...,The Acting Decision references Pensacola Teleg...
2,92291,90600,Telegraph Co. v. Texas (1882),Mentioned,"In Western Union Telegraph Company v. Texas, 1...",The Acting Decision cites Telegraph Co. v. Tex...
3,92291,91869,Robbins v. Shelby County Taxing District (1887),Mentioned,It comes plainly within the principle of the d...,The Acting Decision references Robbins v. Shel...
4,92291,91985,Philadelphia & Southern Steamship Co. v. Penns...,Mentioned,It comes plainly within the principle of the d...,The Acting Decision cites Philadelphia & South...
...,...,...,...,...,...,...
1094,118273,2261846,Mille Lacs Band of Chippewa Indians v. Minneso...,Mentioned,"With respect to the 1850 Executive Order, the ...",The Acting Decision summarizes the District Co...
1095,118273,105018,Youngstown Sheet & Tube Co. v. Sawyer (1952),Mentioned,The Court of Appeals began its analysis of thi...,Youngstown is cited for a general legal princi...
1096,118273,98773,New York Ex Rel. Kennedy v. Becker (1916),Mentioned,"In New York ex rel. Kennedy v. (1916), the Cou...",Kennedy is discussed as an example of differen...
1097,118273,103642,Tulee v. Washington (1942),Mentioned,The conservation necessity standard appears to...,Tulee is cited as the origin of the conservati...


In [8]:
raw_results_df["input_tokens"].replace('', 0).astype(int).sum()

np.int64(2211492)

In [9]:
raw_results_df["output_tokens"].replace('', 0).astype(int).sum()

np.int64(188495)

In [10]:
raw_results_df.to_csv("predictions/gpt_raw_results.csv", index=False)
parsed_results_df.to_csv("predictions/gpt_parsed_results_df.csv", index=False)

In [11]:
parsed_results_df["prediction"].isna().sum()

np.int64(0)

In [12]:
parsed_results_df["prediction"].value_counts()

prediction
Mentioned        778
Overruled        135
Distinguished     93
Limited           33
Reversed          26
Criticized        21
Questioned         9
Affirmed           4
Name: count, dtype: int64

In [13]:
raw_results_df["citing_cluster_id"].nunique()

133

In [14]:
parsed_results_df["citing_cluster_id"].nunique()

133

In [15]:
errored_citing_ids = set(raw_results_df["citing_cluster_id"].unique()) - set(parsed_results_df["citing_cluster_id"].unique())
errored_citing_ids

set()

In [16]:
prediction_df = parsed_results_df

In [17]:
set(df["citing_cluster_id"].unique()) - set(prediction_df["citing_cluster_id"].unique())

set()

In [18]:
prediction_df.to_csv("predictions/gpt_prediction_df.csv", index=False)